In [1]:
import training
import training_constants as tc
import preprocessing as pre
from sklearn.pipeline import Pipeline
import feature_selection as fs
import evaluation

In [2]:
X, y = pre.load_data("/Users/patricia/Documents/code/python/behavior-detection/data/new_logs_labels.csv")
y.head()

<class 'pandas.core.frame.DataFrame'>
Index: 5489 entries, 0 to 5524
Columns: 336 entries, id_log to misc_EqSec_OpFrac_MMC_MtNumerador_total
dtypes: float64(82), int64(253), object(1)
memory usage: 14.1+ MB
<class 'pandas.core.series.Series'>
Index: 5489 entries, 0 to 5524
Series name: comportamento
Non-Null Count  Dtype 
--------------  ----- 
5489 non-null   object
dtypes: object(1)
memory usage: 85.8+ KB


0    ON TASK
1    ON TASK
2    ON TASK
3    ON TASK
4    ON TASK
Name: comportamento, dtype: object

In [3]:
test_size = 0.2  # 80% for training, 20% for testing
X_train, X_test, y_train, y_test = pre.split_train_test_data(X, y, test_size, random_state=42)


Tamanho do conjunto de treino: 4391
Tamanho do conjunto de teste: 1098


In [4]:
import pandas as pd

print("Nro de instancias de cada classe em y_train:\n")
print(pd.Series(y_train).value_counts())
print("\n\nNro de instancias de cada classe em y_test:\n")
print(pd.Series(y_test).value_counts())

Nro de instancias de cada classe em y_train:

ON TASK                 2553
ON SYSTEM                721
OFF TASK                 495
ON TASK CONVERSATION     321
ON TASK OUT              301
Name: count, dtype: int64


Nro de instancias de cada classe em y_test:

ON TASK                 606
ON SYSTEM               186
OFF TASK                134
ON TASK CONVERSATION     93
ON TASK OUT              79
Name: count, dtype: int64


In [5]:
y_train, label_encoder = pre.encode_labels(y_train)
y_test = label_encoder.transform(y_test)

In [6]:
# Pré-processar os dados uma vez
preprocessor = pre.create_preprocessor(X_train)
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [7]:
print(X_train_preprocessed[:5])

[[0.65819387 0.75862069 0.77777778 ... 0.         0.         0.        ]
 [0.99729108 0.68965517 0.55555556 ... 1.         0.         0.        ]
 [0.99854326 0.89655172 0.         ... 1.         0.         0.        ]
 [0.99462421 0.20689655 0.88888889 ... 1.         0.         0.        ]
 [0.99504288 0.27586207 1.         ... 1.         0.         0.        ]]


In [8]:
print(X_train_preprocessed.shape)
import pandas as pd

# Supondo que 'X_train_preprocessed' seja seu numpy.ndarray
df = pd.DataFrame(X_train_preprocessed)

# Agora você pode chamar .describe() no DataFrame
print(df.describe())

(4391, 346)
               0            1            2            3            4    \
count  4391.000000  4391.000000  4391.000000  4391.000000  4391.000000   
mean      0.781210     0.495080     0.498823     0.494901     0.329507   
std       0.311243     0.296815     0.322054     0.316759     0.199229   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.576877     0.241379     0.222222     0.222222     0.160142   
50%       0.994437     0.482759     0.444444     0.444444     0.323843   
75%       0.997089     0.758621     0.777778     0.777778     0.485765   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

               5            6            7            8            9    ...  \
count  4391.000000  4391.000000  4391.000000  4391.000000  4391.000000  ...   
mean      0.053974     0.003872     0.066044     0.044669     0.028543  ...   
std       0.225992     0.062108     0.248388     0.099459     0.088200  ...   
min  

In [9]:
print("y_train: ", y_train[:5])

y_train:  [1 2 2 0 2]


In [10]:
# Creating selectors dynamically
selectors = fs.create_selectors_from_map(X_train_preprocessed, y_train, fs.SELECTOR_MAP)
    
best_selector, best_name, best_params, best_score = fs.evaluate_multiple_feature_selectors_with_search(X_train_preprocessed, y_train, selectors, cv=10)
print(f"Best selector: {best_name} with params: {best_params} and score: {best_score}")

print(f"Best selector: {best_selector}")
print(f"Best params: {best_params}")
print(f"Best score: {best_score}")

# Extrair o melhor número de features
if 'feature_selection__n_features' in best_params:
    n_features_to_select = best_params['feature_selection__n_features']
elif 'feature_selection__n_components' in best_params:
    n_features_to_select = best_params['feature_selection__n_components']
else:
    n_features_to_select = None  # Outros métodos podem não ter n_features

print(f"Best number of features: {n_features_to_select}")


Best selector: rfe with params: {'feature_selection__n_features_to_select': 40} and score: 0.8375007660255381
Best selector: RFE(estimator=RandomForestClassifier(random_state=42), n_features_to_select=40)
Best params: {'feature_selection__n_features_to_select': 40}
Best score: 0.8375007660255381
Best number of features: None


In [11]:
# Adicionar o melhor seletor de variáveis no pipeline de treinamento
base_pipeline = Pipeline([
    ('feature_selection', best_selector)
])

In [12]:
# Bayesian optimization não está funcionando corretamente. Usar Random Search ou Grid Search
cv=10
trained_models = training.train_model(X_train_preprocessed, y_train, tc.BAYESIAN_OPTIMIZATION, base_pipeline, n_iter=100, cv=cv)



Training and evaluating Logistic Regression with Bayesian Optimization:


/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

Bayesian Optimization Best Result for Logistic Regression: 0.6721400964852557

Training and evaluating Decision Tree with Bayesian Optimization:


/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [None, 1, 2] before, using random point [30, 4, 12]
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [None, 1, 2] before, using random point [3, 3, 16]
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [None, 1, 2] before, using random point [30, 3, 2]
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point

Bayesian Optimization Best Result for Decision Tree: 0.8030873089104782

Training and evaluating Random Forest with Bayesian Optimization:


...

Bayesian Optimization Best Result for SVM: 0.7605719225997687

Training and evaluating KNN with Bayesian Optimization:


/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['euclidean', 3, 'distance'] before, using random point ['euclidean', 9, 'uniform']
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 3, 'distance'] before, using random point ['minkowski', 5, 'distance']
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 3, 'distance'] before, using random point ['minkowski', 16, 'distance']
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['manhattan', 3, 'distance'] before, using random point ['euclidean', 14, 'distance']
  warnin

Bayesian Optimization Best Result for KNN: 0.7945317456047148

Training and evaluating XGBoost with Bayesian Optimization:


/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Bayesian Optimization Best Result for XGBoost: 0.8492013588597038


In [13]:
dirpath = "/Users/patricia/Documents/code/python/behavior-detection/output/"

reports = evaluation.generate_reports(trained_models, X_train_preprocessed, y_train, X_test_preprocessed, y_test)
print(evaluation.print_reports(reports, dirpath))
evaluation.save_reports_to_csv(reports, dirpath)


Evaluating Logistic Regression with Bayesian Optimization:
Hiperparâmetros: {'memory': None, 'steps': [('classifier', LogisticRegression(max_iter=5000))], 'verbose': False, 'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 1.0, 'classifier__class_weight': None, 'classifier__dual': False, 'classifier__fit_intercept': True, 'classifier__intercept_scaling': 1, 'classifier__l1_ratio': None, 'classifier__max_iter': 5000, 'classifier__multi_class': 'auto', 'classifier__n_jobs': None, 'classifier__penalty': 'l2', 'classifier__random_state': None, 'classifier__solver': 'lbfgs', 'classifier__tol': 0.0001, 'classifier__verbose': 0, 'classifier__warm_start': False}

Training set report:
Class 0 - Precision: 0.75, Recall: 0.48, F1-Score: 0.59, Support: 495.0
Class 1 - Precision: 0.85, Recall: 0.76, F1-Score: 0.8, Support: 721.0
Class 2 - Precision: 0.83, Recall: 0.92, F1-Score: 0.88, Support: 2553.0
Class 3 - Precision: 0.79, Recall: 0.79, F1-Score: 0.79, Support: 321.0
Class 4 - 